In [9]:
"""
Jorge Manzano Anchelergues y Jaime Usero Aranda
"""
import numpy as np
import pandas as pd
import re
import nltk
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Sacamos los datos del csv
comentarios = pd.read_csv("sentimientos.csv")

# Filtrar los comentarios negativos (sentimiento == 0)
negativo = comentarios[comentarios['sentimiento'] == 0]

# Filtrar los comentarios positivos (sentimiento == 1)
positivo = comentarios[comentarios['sentimiento'] == 1]

# Sacar el tamaño minimo de los dataframes
min_size = min(len(positivo), len(negativo))

# Recortar ambos DataFrames al tamaño mínimo
positivo = positivo.sample(n=len(negativo), random_state=42)
negativo = negativo.sample(n=len(negativo), random_state=42)

# Limpiar los datos
comentariosFiltrados = np.concatenate((positivo['comentario'].values, negativo['comentario'].values))
sentimientos = np.concatenate((positivo['sentimiento'].values, negativo['sentimiento'].values))

# Crear el vectorizador TF-IDF con parámetros adicionales
try:
  stopwords_spanish = stopwords.words('spanish')
except:
  nltk.download('stopwords')

tfidf_vectorizer = TfidfVectorizer(
    max_features=2500,             # Limitar a las 2500 palabras más importantes
    min_df=7,                      # Filtrar palabras que aparecen en menos de 7 documentos
    max_df=0.8,                    # Filtrar palabras que aparecen en más del 80% de los documentos
    stop_words=stopwords_spanish   # Eliminar las palabras comunes en español
)

# Ajustar y transformar los comentarios al modelo TF-IDF
TfModel = tfidf_vectorizer.fit_transform(comentariosFiltrados).toarray()

# Dividir los datos entre entrenamiento y testeo
X_train, X_test, y_train, y_test = train_test_split(TfModel, sentimientos, test_size=0.2, random_state=0)

# Entrenamos al modelo
text_classifier = RandomForestClassifier(n_estimators=200, random_state=0)
text_classifier.fit(X_train, y_train)

# Predicciones
predictions = text_classifier.predict(X_test)

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[381  49]
 [ 59 414]]
              precision    recall  f1-score   support

           0       0.87      0.89      0.88       430
           1       0.89      0.88      0.88       473

    accuracy                           0.88       903
   macro avg       0.88      0.88      0.88       903
weighted avg       0.88      0.88      0.88       903

0.8803986710963455
